# Hi-C Scaffolding Manuka


In [1]:
#Set Up
WKDIR=/workspace/hraijc/HiC_trials/Manuka

mkdir -p $WKDIR/log
cd $WKDIR

In [3]:
# Input fastq files need to be gzipped for hic-scaffolding-pipeline LOL
# From here: /input/genomic/plant/Leptospermum/scoparium/CrimsonGlory/Genome/HiC
sbatch << EOF
#!/bin/bash
#SBATCH -J gzip
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=4G
#SBATCH --time=23:00:00

cd $WKDIR

gzip manuka_S3HiC_AD007_R1.fastq -c > manuka_S3HiC_AD007_R1.fastq.gz
gzip manuka_S3HiC_AD007_R2.fastq -c > manuka_S3HiC_AD007_R2.fastq.gz



EOF


Submitted batch job 1522901


In [4]:
#IN INTERACTIVE SESSION:
#Use Srun not Salloc
"""
srun --time 01:00:00 --cpus-per-task 1 --mem-per-cpu 4G --pty bash
#WKDIR=/workspace/hraijc/HiC_trials/HiC23/NovaSeq/Urchin
cd $WKDIR
git clone git@github.com:ignacio3437/hic-scaffolding-pipeline.git
cd ${WKDIR}/hic-scaffolding-pipeline/
git checkout dev
cd ${WKDIR}/hic-scaffolding-pipeline/containers
sudo -E singularity build pipeline_container.sif pipeline_container.def

"""

In [2]:
# Modify Config file

cat ${WKDIR}/hic-scaffolding-pipeline/nextflow.config

includeConfig './conf/base.config'

params {

    // R1, R2 dataset
<<<<<<< HEAD
    reads_R1                = "/workspace/hraijc/HiC_trials/Manuka/manuka_S3HiC_AD007_R1.fastq.gz"
    reads_R2                = "/workspace/hraijc/HiC_trials/Manuka/manuka_S3HiC_AD007_R1.fastq.gz"

    // Assembly file
    assembly_fasta          = "/workspace/hraijc/HiC_trials/Manuka/primary.genome.scf.fasta" // SALSA requires fasta to not contain ":" character

    // Prefix for output files
    out_prefix              = "manuka_V1"
    reads_R1                = "/workspace/hraijc/TEMP/Myna_HiC23_R1.fastq.gz"
    reads_R2                = "/workspace/hraijc/TEMP/Myna_HiC23_R2.fastq.gz"

    // Assembly file
    assembly_fasta          = "/workspace/hraijc/HiC_trials/HiC23/Assemblies/AcTris_vAus2_1.fasta" // SALSA requires fasta to not contain ":" character

    // Prefix for output files
    out_prefix              = "AcTris_vAus2_1_hic"
>>>>>>> ed7941db422685e0243c4558d4930556b1569a65

    //STAG 1 
  

In [14]:
# Setup slurm.sh

cat << EOF > hic_pipeline_slurm.sh
#!/bin/bash -e

#SBATCH -J hic_pipeline_${USER}
#SBATCH --time=170:00:00
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --mem=4G

ml nextflow/22.10.4
ml apptainer/1.1

cd ${WKDIR}/hic-scaffolding-pipeline

srun nextflow main.nf -resume
EOF





In [4]:
# Run Pipeline
#chmod u+x ./hic_pipeline_slurm.sh
sbatch ./hic_pipeline_slurm.sh

Submitted batch job 1529297



## Just redo YAHS



In [10]:
# Set Variables
WKDIR=/workspace/hraijc/HiC_trials/Manuka
ASSEMBLY=${WKDIR}/primary.genome.scf.fasta
BAMFILE=${WKDIR}/manuka_V1_dedup.bam
APREFIX=manuka_V1
cd ${WKDIR}

In [2]:
mkdir -p ${WKDIR}/log

In [3]:
# Starting with Bamfile generated with HiC-scaffolding-pipeline that keeps stalling.
ln -s ${WKDIR}/hic-scaffolding-pipeline/results/bam/manuka_V1_dedup.bam ${BAMFILE}


ln: failed to create symbolic link ‘/workspace/hraijc/HiC_trials/Manuka/manuka_V1_dedup.bam’: File exists


: 1

In [4]:
#Just run YAHS

sbatch << EOF
#!/bin/bash
#SBATCH -J YAHS
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=48G
#SBATCH --time=23:00:00

cd ${WKDIR}

module load samtools

samtools faidx ${ASSEMBLY}


### Run YAHS ###
#/workspace/hraijc/git_clones/yahs/yahs ${ASSEMBLY} ${BAMFILE} -o ${APREFIX}_yahs #This is actually _yahs_20k because it runs out of RAM silently. 
/workspace/hraijc/git_clones/yahs/yahs ${ASSEMBLY} ${BAMFILE} --no-mem-check -o ${APREFIX}_yahs_20K -r 20000,50000,100000,200000,500000,1000000,2000000,5000000,10000000,20000000,50000000,100000000,200000000,500000000
/workspace/hraijc/git_clones/yahs/yahs ${ASSEMBLY} ${BAMFILE} --no-mem-check -o ${APREFIX}_yahs_10K -r 10000,20000,50000,100000,200000,500000,1000000,2000000,5000000,10000000,20000000,50000000,100000000,200000000,500000000
/workspace/hraijc/git_clones/yahs/yahs ${ASSEMBLY} ${BAMFILE} --no-mem-check -o ${APREFIX}_yahs_5K -r 5000,10000,20000,50000,100000,200000,500000,1000000,2000000,5000000,10000000,20000000,50000000,100000000,200000000,500000000
/workspace/hraijc/git_clones/yahs/yahs ${ASSEMBLY} ${BAMFILE} --no-mem-check -o ${APREFIX}_yahs_1K -r 1000,5000,10000,20000,50000,100000,200000,500000,1000000,2000000,5000000,10000000,20000000,50000000,100000000,200000000,500000000




### YAHS without error correction. ###
#/workspace/hraijc/git_clones/yahs/yahs --no-contig-ec --no-scaffold-ec $ASSEMBLY ${BAMFILE} -o ${APREFIX}_yahsNoEC




EOF


Submitted batch job 1554381


In [8]:
out=${APREFIX}_yahs_20K # prefix of outfiles produced by YAHS.  
contigs=${ASSEMBLY} # need to be indexed, i.e., test.fa.gz.fai in same directory
wkdir=${WKDIR}


sbatch --job-name=hic_mapyahs \
    -o ${WKDIR}/log/%J.out \
    -e ${WKDIR}/log/%J.err \
    --cpus-per-task=8 \
    --mem=24G \
    --time=03:10:00 \
    --export=out=$out,contigs=$contigs,wkdir=$wkdir \
    /workspace/hraijc/Gitrepos/High-quality-genomes/Methods/DNase_HiC/notebooks/yahs_contactmapgen2.sh

Submitted batch job 1555370


In [12]:
sbatch << EOF
#!/bin/bash
#SBATCH -J YAHS
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=4G
#SBATCH --time=2:00:00

module load samtools


cd ${WKDIR}

module load samtools

/workspace/hraijc/git_clones/yahs/juicer post -o ${APREFIX}_v2 ${APREFIX}_yahs_10K_JBAT.review.assembly ${APREFIX}_yahs_10K_JBAT.liftover.agp ${ASSEMBLY}



EOF

Submitted batch job 1560483


## QC Assemblies

In [13]:


#QC assemblies before Hi-Scaffolding


module load BBMap/38.33
statswrapper.sh in=$ASSEMBLY,${APREFIX}_yahs_1K_scaffolds_final.fa,${APREFIX}_yahs_5K_scaffolds_final.fa,${APREFIX}_yahs_10K_scaffolds_final.fa,${APREFIX}_yahs_20K_scaffolds_final.fa,manuka_v2.fa format=3
module unload BBMap/38.33


java -ea -Xmx200m -cp /software/bioinformatics/BBMap-38.33/current/ jgi.AssemblyStatsWrapper format=3 in=/workspace/hraijc/HiC_trials/Manuka/primary.genome.scf.fasta,manuka_V1_yahs_1K_scaffolds_final.fa,manuka_V1_yahs_5K_scaffolds_final.fa,manuka_V1_yahs_10K_scaffolds_final.fa,manuka_V1_yahs_20K_scaffolds_final.fa,manuka_v2.fa format=3
n_scaffolds	n_contigs	scaf_bp	contig_bp	gap_pct	scaf_N50	scaf_L50	ctg_N50	ctg_L50	scaf_N90	scaf_L90	ctg_N90	ctg_L90	scaf_max	ctg_max	scaf_n_gt50K	scaf_pct_gt50K	gc_avg	gc_std	filename
14133	14149	351338343	351336743	0.000	352	229082	358	227892	2341	15286	2355	15205	2483064	2483064	1352	81.891	0.40979	0.06297	/powerplant/workspace/hraijc/HiC_trials/Manuka/primary.genome.scf.fasta
12356	14601	351784143	351336743	0.127	33	2920380	448	192865	646	19924	2636	14438	12101987	2483064	369	87.536	0.40979	0.05691	/powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_V1_yahs_1K_scaffolds_final.fa
12681	14601	351719143	351336743	0.109	7	13688648	448	192865	732	15789	2

| n_scaffolds 	| n_contigs 	| scaf_bp 	| contig_bp 	| gap_pct 	| scaf_N50 	| scaf_L50 	| ctg_N50 	| ctg_L50 	| scaf_N90 	| scaf_L90 	| ctg_N90 	| ctg_L90 	| scaf_max 	| ctg_max 	| scaf_n_gt50K 	| scaf_pct_gt50K 	| gc_avg 	| gc_std 	| filename 	|
|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|
| 14133 	| 14149 	| 351338343 	| 351336743 	| 0.000 	| 352 	| 229082 	| 358 	| 227892 	| 2341 	| 15286 	| 2355 	| 15205 	| 2483064 	| 2483064 	| 1352 	| 81.891 	| 0.40979 	| 0.06297 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/primary.genome.scf.fasta 	|
| 12356 	| 14601 	| 351784143 	| 351336743 	| 0.127 	| 33 	| 2920380 	| 448 	| 192865 	| 646 	| 19924 	| 2636 	| 14438 	| 12101987 	| 2483064 	| 369 	| 87.536 	| 0.40979 	| 0.05691 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_V1_yahs_1K_scaffolds_final.fa 	|
| 12681 	| 14601 	| 351719143 	| 351336743 	| 0.109 	| 7 	| 13688648 	| 448 	| 192865 	| 732 	| 15789 	| 2636 	| 14438 	| 48386517 	| 2483064 	| 225 	| 86.240 	| 0.40979 	| 0.05741 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_V1_yahs_5K_scaffolds_final.fa 	|
| 12920 	| 14601 	| 351671343 	| 351336743 	| 0.095 	| 7 	| 19256392 	| 448 	| 192865 	| 955 	| 14454 	| 2636 	| 14438 	| 36412886 	| 2483064 	| 217 	| 84.894 	| 0.40979 	| 0.05826 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_V1_yahs_10K_scaffolds_final.fa 	|
| 13280 	| 14601 	| 351599343 	| 351336743 	| 0.075 	| 7 	| 19961310 	| 448 	| 192865 	| 1315 	| 14454 	| 2636 	| 14438 	| 43133744 	| 2483064 	| 290 	| 82.355 	| 0.40979 	| 0.05976 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_V1_yahs_20K_scaffolds_final.fa 	|
| 12924 	| 14601 	| 351670543 	| 351336743 	| 0.095 	| 8 	| 15777695 	| 448 	| 192865 	| 959 	| 14454 	| 2636 	| 14438 	| 31070159 	| 2483064 	| 221 	| 84.894 	| 0.40979 	| 0.05828 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2.fa 	|

| n_scaffolds 	| n_contigs 	| scaf_bp 	| contig_bp 	| gap_pct 	| scaf_N50 	| scaf_L50 	| ctg_N50 	| ctg_L50 	| scaf_N90 	| scaf_L90 	| ctg_N90 	| ctg_L90 	| scaf_max 	| ctg_max 	| scaf_n_gt50K 	| scaf_pct_gt50K 	| gc_avg 	| gc_std 	| filename 	|
|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|
| 14133 	| 14149 	| 351338343 	| 351336743 	| 0.000 	| 352 	| 229082 	| 358 	| 227892 	| 2341 	| 15286 	| 2355 	| 15205 	| 2483064 	| 2483064 	| 1352 	| 81.891 	| 0.40979 	| 0.06297 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/primary.genome.scf.fasta 	|
| 12924 	| 14601 	| 351670543 	| 351336743 	| 0.095 	| 8 	| 15777695 	| 448 	| 192865 	| 959 	| 14454 	| 2636 	| 14438 	| 31070159 	| 2483064 	| 221 	| 84.894 	| 0.40979 	| 0.05828 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2.fa 	|

In [1]:
WKDIR=/workspace/hraijc/HiC_trials/Manuka
cd $WKDIR

In [2]:
sbatch << EOF
#!/bin/bash -e


#SBATCH --job-name ASM_QC
#SBATCH --time=10:00:00
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --mem=4G

cd /workspace/hraijc/HiC_trials/Manuka/assembly_qc

ml unload perl
ml Python/3.10.4-GCCcore-11.2.0-bare
ml apptainer/1.1
ml nextflow/22.10.4
ml assemblathon_stats/14dfdab

export TMPDIR="/workspace/$USER/tmp"

srun nextflow main.nf -profile slurm -resume


EOF

Submitted batch job 1663385


## BLOBTOOLS

In [8]:
WKDIR=/workspace/hraijc/HiC_trials/Manuka/blobplots

ASSEMBLY=/workspace/hraijc/HiC_trials/Manuka/manuka_v2.fa

READS=/workspace/hraijc/HiC_trials/Manuka/manuka_S3HiC_AD007_R1.fastq
MINIMAPX=sr
PREFIX=ManukaV2

cd ${WKDIR}

In [13]:
mkdir -p ${WKDIR}/log
mkdir -p ${WKDIR}/blasthits
mkdir -p ${WKDIR}/bamfiles

In [18]:
# Create coverage files.
sbatch << EOF
#!/bin/bash
#SBATCH -J minimap_samtools
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=32
#SBATCH --mem=20G
#SBATCH --time=11:00:00

module load minimap2/2.22
module load samtools/1.16

## Map ##
minimap2 -t 32 -ax ${MINIMAPX} ${ASSEMBLY} ${READS} > ${WKDIR}/bamfiles/${PREFIX}.sam

## Sam -> bam ##
samtools view --threads 32 -b ${WKDIR}/bamfiles/${PREFIX}.sam > ${WKDIR}/bamfiles/${PREFIX}.bam

## Sort bam ##
samtools sort --threads 32 ${WKDIR}/bamfiles/${PREFIX}.bam > ${WKDIR}/bamfiles/${PREFIX}.sorted.bam

## index bam ##
samtools index ${WKDIR}/bamfiles/${PREFIX}.sorted.bam


EOF


Submitted batch job 1620062


In [ ]:
cd ${WKDIR}/bamfiles/
mkdir sorted
mv *.sorted.bam* sorted
rm -rf *am
mv sorted/* .
rm -rf sorted/

In [15]:
# Create Blast files.


sbatch << EOF
#!/bin/bash
#SBATCH -J megablast
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=32
#SBATCH --mem=10G
#SBATCH --time=11:00:00

module load ncbi-blast/2.11.0

blastn \
-task megablast \
-query ${ASSEMBLY} \
-db /workspace/ComparativeDataSources/NCBI/nt/nt \
-outfmt '6 qseqid staxids bitscore std' \
-max_target_seqs 1 \
-max_hsps 1 \
-num_threads 32 \
-evalue 1e-25 \
-out ${WKDIR}/blasthits/${PREFIX}.vs.nt.mts1.hsp1.1e25.megablast.out




EOF



Submitted batch job 1620060


In [20]:
sbatch << EOF
#!/bin/bash
#SBATCH -J blobtools
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=2
#SBATCH --mem=1G
#SBATCH --time=2:00:00

module load conda
conda activate hraijc_blobtools
/workspace/hraijc/git_clones/blobtools/blobtools --version

/workspace/hraijc/git_clones/blobtools/blobtools create \
 -i ${ASSEMBLY} \
 -b ${WKDIR}/bamfiles/${PREFIX}.sorted.bam \
 -t ${WKDIR}/blasthits/${PREFIX}.vs.nt.mts1.hsp1.1e25.megablast.out \
 -o ${WKDIR}/${PREFIX}.blobplot

 
 
/workspace/hraijc/git_clones/blobtools/blobtools plot \
 -i ${WKDIR}/${PREFIX}.blobplot.blobDB.json \
 -o ${WKDIR}/${PREFIX}.blobplot
 
 
 
/workspace/hraijc/git_clones/blobtools/blobtools view \
 -i ${WKDIR}/${PREFIX}.blobplot.blobDB.json \
 -o ${WKDIR}/${PREFIX}.blobplot


EOF

Submitted batch job 1620158


## AFTER BLOBPLOT FILTERING

In [9]:
module load seqtk
seqtk subseq $ASSEMBLY /workspace/hraijc/HiC_trials/Manuka/blobplots/blobplot_keep.txt > /workspace/hraijc/HiC_trials/Manuka/manuka_v2a.fa

In [2]:
#Set Variables
WKDIR=/workspace/hraijc/HiC_trials/Manuka/
ASSEMBLY=/workspace/hraijc/HiC_trials/Manuka/manuka_v2a.fa
APREFIX=manuka_v2b
HiC_RAW=/workspace/hraijc/HiC_trials/Manuka/manuka_S3HiC_AD007_R
READ1=${HiC_RAW}1.fastq
READ2=${HiC_RAW}2.fastq
CPUS=1
BAMFILE=${APREFIX}_presort_marked.bam

In [14]:
du -sh $READ1
du -sh $READ2
du -sh $ASSEMBLY


24K	/workspace/hraijc/HiC_trials/Manuka/manuka_S3HiC_AD007_R1.fastq
24K	/workspace/hraijc/HiC_trials/Manuka/manuka_S3HiC_AD007_R2.fastq
322M	/workspace/hraijc/HiC_trials/Manuka/manuka_v2a.fa


In [15]:
#Run HiC workflow again after doing blobtools filtering.

sbatch << EOF
#!/bin/bash
#SBATCH -J hic
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=${CPUS}
#SBATCH --mem=25G
#SBATCH --time=45:00:00

module load bwa/0.7.17
module load samtools/1.16

cd $WKDIR

###Align reads with BWA. Use -5SP for Hi-C reads.###
#bwa index ${ASSEMBLY}
#bwa mem -5SP -t ${CPUS} ${ASSEMBLY} ${READ1} ${READ2} -o ${APREFIX}.sam



###Flag PCR Duplicates with SAMBLASTER###
#/workspace/hraijc/git_clones/samblaster/samblaster -i ${APREFIX}.sam -o ${APREFIX}_marked_byread.sam



### Remove unmmaped and non-primary aligned reads. Sort and Index bam files.### 
#samtools view --threads ${CPUS} -S -b -h -F 2316 ${APREFIX}_marked_byread.sam > ${BAMFILE}
samtools faidx ${ASSEMBLY}

### Run YAHS ###
/workspace/hraijc/git_clones/yahs/yahs ${ASSEMBLY} ${BAMFILE} --no-mem-check -o ${APREFIX}_yahs_20K -r 20000,50000,100000,200000,500000,1000000,2000000,5000000,10000000,20000000,50000000,100000000,200000000,500000000
/workspace/hraijc/git_clones/yahs/yahs ${ASSEMBLY} ${BAMFILE} --no-mem-check -o ${APREFIX}_yahs_10K -r 10000,20000,50000,100000,200000,500000,1000000,2000000,5000000,10000000,20000000,50000000,100000000,200000000,500000000
/workspace/hraijc/git_clones/yahs/yahs ${ASSEMBLY} ${BAMFILE} --no-mem-check -o ${APREFIX}_yahs_5K -r 5000,10000,20000,50000,100000,200000,500000,1000000,2000000,5000000,10000000,20000000,50000000,100000000,200000000,500000000




EOF


Submitted batch job 1663810


In [3]:
# QC ASSEMBLIES

cd $WKDIR
module load BBMap/38.33
statswrapper.sh in=manuka_v2.fa,manuka_v2a.fa,${APREFIX}_yahs_5K_scaffolds_final.fa,${APREFIX}_yahs_10K_scaffolds_final.fa,${APREFIX}_yahs_20K_scaffolds_final.fa format=3
module unload BBMap/38.33


java -ea -Xmx200m -cp /software/bioinformatics/BBMap-38.33/current/ jgi.AssemblyStatsWrapper format=3 in=manuka_v2.fa,manuka_v2a.fa,manuka_v2b_yahs_5K_scaffolds_final.fa,manuka_v2b_yahs_10K_scaffolds_final.fa,manuka_v2b_yahs_20K_scaffolds_final.fa format=3
n_scaffolds	n_contigs	scaf_bp	contig_bp	gap_pct	scaf_N50	scaf_L50	ctg_N50	ctg_L50	scaf_N90	scaf_L90	ctg_N90	ctg_L90	scaf_max	ctg_max	scaf_n_gt50K	scaf_pct_gt50K	gc_avg	gc_std	filename
12924	14601	351670543	351336743	0.095	8	15777695	448	192865	959	14454	2636	14438	31070159	2483064	221	84.894	0.40979	0.05828	/powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2.fa
462	2052	294318832	294002332	0.108	6	21727876	318	244159	17	4623713	1218	66000	31070159	2483064	139	97.540	0.39648	0.01118	/powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2a.fa
406	2117	294343032	294002332	0.116	6	22218532	321	237260	14	7933626	1228	65584	38879479	2483064	113	98.088	0.39648	0.01758	/powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2b_yahs_5K_s

| n_scaffolds 	| n_contigs 	| scaf_bp 	| contig_bp 	| gap_pct 	| scaf_N50 	| scaf_L50 	| ctg_N50 	| ctg_L50 	| scaf_N90 	| scaf_L90 	| ctg_N90 	| ctg_L90 	| scaf_max 	| ctg_max 	| scaf_n_gt50K 	| scaf_pct_gt50K 	| gc_avg 	| gc_std 	| filename 	|
|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|
| 12924 	| 14601 	| 351670543 	| 351336743 	| 0.095 	| 8 	| 15777695 	| 448 	| 192865 	| 959 	| 14454 	| 2636 	| 14438 	| 31070159 	| 2483064 	| 221 	| 84.894 	| 0.40979 	| 0.05828 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2.fa 	|
| 462 	| 2052 	| 294318832 	| 294002332 	| 0.108 	| 6 	| 21727876 	| 318 	| 244159 	| 17 	| 4623713 	| 1218 	| 66000 	| 31070159 	| 2483064 	| 139 	| 97.540 	| 0.39648 	| 0.01118 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2a.fa 	|
| 406 	| 2117 	| 294343032 	| 294002332 	| 0.116 	| 6 	| 22218532 	| 321 	| 237260 	| 14 	| 7933626 	| 1228 	| 65584 	| 38879479 	| 2483064 	| 113 	| 98.088 	| 0.39648 	| 0.01758 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2b_yahs_5K_scaffolds_final.fa 	|
| 457 	| 2117 	| 294332832 	| 294002332 	| 0.112 	| 5 	| 24358461 	| 321 	| 237260 	| 12 	| 7933626 	| 1228 	| 65584 	| 39989661 	| 2483064 	| 113 	| 97.745 	| 0.39648 	| 0.01721 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2b_yahs_10K_scaffolds_final.fa 	|
| 477 	| 2117 	| 294328832 	| 294002332 	| 0.111 	| 6 	| 22204332 	| 321 	| 237260 	| 13 	| 8558000 	| 1228 	| 65584 	| 33501919 	| 2483064 	| 110 	| 97.522 	| 0.39648 	| 0.01708 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2b_yahs_20K_scaffolds_final.fa 	|

In [4]:
out=${APREFIX}_yahs_10K # prefix of outfiles produced by YAHS.  
contigs=${ASSEMBLY} # need to be indexed, i.e., test.fa.gz.fai in same directory
wkdir=${WKDIR}


sbatch --job-name=hic_mapyahs \
    -o ${WKDIR}/log/%J.out \
    -e ${WKDIR}/log/%J.err \
    --cpus-per-task=8 \
    --mem=24G \
    --time=03:10:00 \
    --export=out=$out,contigs=$contigs,wkdir=$wkdir \
    /workspace/hraijc/Gitrepos/High-quality-genomes/Methods/DNase_HiC/notebooks/yahs_contactmapgen2.sh

Submitted batch job 1685862


In [5]:
sbatch << EOF
#!/bin/bash
#SBATCH -J YAHS
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=4G
#SBATCH --time=2:00:00

module load samtools


cd ${WKDIR}

module load samtools

/workspace/hraijc/git_clones/yahs/juicer post -o ${APREFIX}_v2c ${APREFIX}_yahs_10K_JBAT.review.assembly ${APREFIX}_yahs_10K_JBAT.liftover.agp ${ASSEMBLY}



EOF

Submitted batch job 1685904


In [7]:
#Remove short contigs.
module load seqkit
seqkit seq manuka_v2c.fa -m 100000 > manuka_v2c_100kb.fa
seqkit seq manuka_v2c.fa -m 10000 > manuka_v2c_10kb.fa
module unload seqkit

In [3]:
sbatch << EOF
#!/bin/bash
#SBATCH -J BUSCO
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=12
#SBATCH --mem=4G
#SBATCH --time=2:00:00

module load BUSCO/v5.2.2


cd ${WKDIR}

busco -i manuka_v2c.fa -l eudicots_odb10 -o manuka_v2c_Busco -m geno -c 12
busco -i manuka_v2c_10kb.fa -l eudicots_odb10 -o manuka_v2c10kb_Busco -m geno -c 12
busco -i manuka_v2c_100kb.fa -l eudicots_odb10 -o manuka_v2c100kb_Busco -m geno -c 12


EOF

Submitted batch job 1707648


In [8]:
cd $WKDIR
module load BBMap/38.33
statswrapper.sh in=manuka_v2.fa,manuka_v2a.fa,manuka_v2c.fa,manuka_v2c_10kb.fa,manuka_v2c_100kb.fa format=3
module unload BBMap/38.33

java -ea -Xmx200m -cp /software/bioinformatics/BBMap-38.33/current/ jgi.AssemblyStatsWrapper format=3 in=manuka_v2.fa,manuka_v2a.fa,manuka_v2c.fa,manuka_v2c_10kb.fa,manuka_v2c_100kb.fa format=3
n_scaffolds	n_contigs	scaf_bp	contig_bp	gap_pct	scaf_N50	scaf_L50	ctg_N50	ctg_L50	scaf_N90	scaf_L90	ctg_N90	ctg_L90	scaf_max	ctg_max	scaf_n_gt50K	scaf_pct_gt50K	gc_avg	gc_std	filename
12924	14601	351670543	351336743	0.095	8	15777695	448	192865	959	14454	2636	14438	31070159	2483064	221	84.894	0.40979	0.05828	/powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2.fa
462	2052	294318832	294002332	0.108	6	21727876	318	244159	17	4623713	1218	66000	31070159	2483064	139	97.540	0.39648	0.01118	/powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2a.fa
513	2197	294337632	294002332	0.114	6	21553676	329	233913	17	1135615	1265	63384	29967390	2483064	129	97.074	0.39648	0.01714	/powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2c.fa
478	2162	294264632	293929332	0.114	6	21553676	329	233913	17	1135615	1

| n_scaffolds 	| n_contigs 	| scaf_bp 	| contig_bp 	| gap_pct 	| scaf_N50 	| scaf_L50 	| ctg_N50 	| ctg_L50 	| scaf_N90 	| scaf_L90 	| ctg_N90 	| ctg_L90 	| scaf_max 	| ctg_max 	| scaf_n_gt50K 	| scaf_pct_gt50K 	| gc_avg 	| gc_std 	| filename 	| BUSCO 	|
|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|---	|
| 12924 	| 14601 	| 351670543 	| 351336743 	| 0.095 	| 8 	| 15777695 	| 448 	| 192865 	| 959 	| 14454 	| 2636 	| 14438 	| 31070159 	| 2483064 	| 221 	| 84.894 	| 0.40979 	| 0.05828 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2.fa 	|  	|
| 462 	| 2052 	| 294318832 	| 294002332 	| 0.108 	| 6 	| 21727876 	| 318 	| 244159 	| 17 	| 4623713 	| 1218 	| 66000 	| 31070159 	| 2483064 	| 139 	| 97.540 	| 0.39648 	| 0.01118 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2a.fa 	| C:97.9%[S:95.2%,D:2.7%],F:1.0%,M:1.1%,n:1614 	|
| 513 	| 2197 	| 294337632 	| 294002332 	| 0.114 	| 6 	| 21553676 	| 329 	| 233913 	| 17 	| 1135615 	| 1265 	| 63384 	| 29967390 	| 2483064 	| 129 	| 97.074 	| 0.39648 	| 0.01714 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2c.fa 	| C:97.9%[S:95.0%,D:2.9%],F:1.0%,M:1.1%,n:1614 	|
| 478 	| 2162 	| 294264632 	| 293929332 	| 0.114 	| 6 	| 21553676 	| 329 	| 233913 	| 17 	| 1135615 	| 1264 	| 63465 	| 29967390 	| 2483064 	| 129 	| 97.098 	| 0.39648 	| 0.01297 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2c_10kb.fa 	| C:97.3%[S:94.2%,D:3.1%],F:0.9%,M:1.8%,n:2326 	|
| 66 	| 1701 	| 280882613 	| 280557013 	| 0.116 	| 6 	| 21553676 	| 301 	| 249577 	| 14 	| 7403826 	| 1096 	| 77283 	| 29967390 	| 2483064 	| 66 	| 100.000 	| 0.39631 	| 0.00965 	| /powerplant/workspace/hraijc/HiC_trials/Manuka/manuka_v2c_100kb.fa 	| C:96.9%[S:94.4%,D:2.5%],F:0.9%,M:2.2%,n:2326 	|

In [4]:
##############
# Subset HiC reads to keep unique mapped reads only. speed up analysis for future iterations.

##############

cd ${WKDIR}

sbatch << EOF
#!/bin/bash
#SBATCH -J samtoolsfastq
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=8
#SBATCH --mem=12G
#SBATCH --time=12:00:00

cd ${WKDIR}

module load samtools 

samtools view -h -b -@ 8 -F 3340 ${BAMFILE} | samtools sort -@ 8 -n | samtools fastq -@ 8 -1 ${OUTPREFIX}_R1.fastq.gz -2 ${OUTPREFIX}_R2.fastq.gz -0 /dev/null -s /dev/null -n



EOF

Submitted batch job 2009244
